<a href="https://colab.research.google.com/github/basselkassem/easy21/blob/master/td_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libs

In [1]:
!pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=c490330e3c0b83dfb04af66e1de57ca553886ac90c3ce8c744a66aa337802c77
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [2]:
from google.colab import drive
drive.mount("/content/drive", )
%cd /content/drive/'My Drive'/'Colab Notebooks'/easy21

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/easy21


# Import Libs

In [0]:
import import_ipynb
import environment as env
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.graph_objects as go
from matplotlib import cm
import plotly.figure_factory as FF
from scipy.spatial import Delaunay

#Implementation of Sarsa(λ):

* Initialise the value function to zero. 
* Use the same step-size and exploration schedules as in Monte-Carlo control. 
* Run the algorithm with parameter values λ = [0, 0.1, 0.2, ..., 1]. 
* Stop each run after 1000 episodes and report the mean-squared error sum(Q(s, a) − Q∗(s, a))2 over all states s and actions a, comparing the true values Q∗(s, a) computed in Monte-Carlo control with the estimated values Q(s, a) computed by Sarsa.
* Plot the meansquared error against λ. For λ = 0 and λ = 1 only, plot the learning curve of mean-squared error against episode number.

# Class Sarsa

In [0]:
class SarsaLambda:

    def __init__(self, td_lambda, N0):
        self.td_lambda = td_lambda
        self.N0 = N0
        self.easy21 = env.Environment(mode = 'no_debug')
        self.Q = defaultdict(int)
        self.E = defaultdict(int)
        self.Nsa = defaultdict(int)

    # compute the number of visiting state s
    def get_Ns(self, state):
        return self.Nsa[state, 0] + self.Nsa[state, 1]
    
    def init_E(self):
        for dealer in env.DEALER_FIRST_CARD_SPACE:
            for player in range(1, env.MAX_VALUE + 1):
                state = (dealer, player)
                self.E[state, env.HIT] = 0
                self.E[state, env.STICK] = 0

    def epsilon_greedy(self, state):
        epsilon = self.N0 / (self.N0 + self.get_Ns(state))
        if np.random.random() < epsilon:
            hit = self.Q[state, 0]
            stick = self.Q[state, 1]
            if hit > stick:
                return 0
            if hit == stick:
                return self.easy21.sample_action()
            if hit < stick:
                return 1
        else:
            return self.easy21.sample_action()

    def agent_policy(self, state):
        return self.epsilon_greedy(state)

    def policy_evaluation(self):
        pass

    def ploicy_imporvment(self):
        pass

    def policy_iteration(self):
        for episode in np.arange(1000):
            self.init_E()
            self.easy21 = env.Environment(mode = 'no_debug')
            state, _, done = self.easy21.init()
            player_action = self.easy21.sample_action()

            done = False
            episode = []
            returned_reward = 0
            while not done:
                state, reward, done = self.easy21.step(state, player_action)
                player_action = self.agent_policy(state)
                #episode.append((state, player_action))
    

In [21]:
agent = SarsaLambda(0.1, 100)
agent.init_E()
agent.E

defaultdict(int,
            {((1, 1), 0): 0,
             ((1, 1), 1): 0,
             ((1, 2), 0): 0,
             ((1, 2), 1): 0,
             ((1, 3), 0): 0,
             ((1, 3), 1): 0,
             ((1, 4), 0): 0,
             ((1, 4), 1): 0,
             ((1, 5), 0): 0,
             ((1, 5), 1): 0,
             ((1, 6), 0): 0,
             ((1, 6), 1): 0,
             ((1, 7), 0): 0,
             ((1, 7), 1): 0,
             ((1, 8), 0): 0,
             ((1, 8), 1): 0,
             ((1, 9), 0): 0,
             ((1, 9), 1): 0,
             ((1, 10), 0): 0,
             ((1, 10), 1): 0,
             ((1, 11), 0): 0,
             ((1, 11), 1): 0,
             ((1, 12), 0): 0,
             ((1, 12), 1): 0,
             ((1, 13), 0): 0,
             ((1, 13), 1): 0,
             ((1, 14), 0): 0,
             ((1, 14), 1): 0,
             ((1, 15), 0): 0,
             ((1, 15), 1): 0,
             ((1, 16), 0): 0,
             ((1, 16), 1): 0,
             ((1, 17), 0): 0,
           